In [1]:
pip install riotwatcher

Note: you may need to restart the kernel to use updated packages.


In [2]:
from riotwatcher import LolWatcher, ApiError
import pandas as pd
import numpy as np

In [3]:
# golbal variables
api_key = 'RGAPI-9dcc4aac-bc2e-40f8-8d52-87068752be5f'
watcher = LolWatcher(api_key)
my_region = 'na1'

me = watcher.summoner.by_name(my_region, 'Fishlord')
print(me)

{'id': 'lMLT3zt1FipxNAqCJP6VH06KCyV4fkGRviX-jqYAFfkhhdI', 'accountId': 'qw_YMvgFYkhftDqCnxwV6CqoJU4oea-D6XduaeWXFZ4pQ1s', 'puuid': 'hpSoRyPAyAxHGjoIWI1ibb8hnDuNBIs6h6o1Swvp4CARR9y9v_iYbM5CCwAZGJOtkvnVt1bCMeDI-g', 'name': 'Fishlord', 'profileIconId': 4079, 'revisionDate': 1677816938152, 'summonerLevel': 646}


In [4]:
# get the latest version of the game from data dragon
versions = watcher.data_dragon.versions_for_region(my_region)
champions_version = versions['n']['champion']

In [5]:
# get current champion info, save as dataframe
current_champ_list = watcher.data_dragon.champions(champions_version)
champ_info_df = pd.DataFrame()
for index, champ in enumerate(current_champ_list['data'].values()): 
  current_champ_info = {'id': champ['id'], 'key': champ['key'], 'title': champ['title'], 'blurb': champ['blurb']}
  current_champ_info.update(champ['info'])
  current_champ_info['tags'] = champ['tags']
  current_champ_info['partype'] = champ['partype']
  current_champ_info.update(champ['stats'])
  current_champ_df = pd.DataFrame.from_dict({index:(current_champ_info)}, orient='index')
  champ_info_df = pd.concat([champ_info_df, current_champ_df], axis=0)
champ_info_df.head()

,id,key,title,blurb,attack,defense,magic,difficulty,tags,partype,...,hpregen,hpregenperlevel,mpregen,mpregenperlevel,crit,critperlevel,attackdamage,attackdamageperlevel,attackspeedperlevel,attackspeed
0,Aatrox,266,the Darkin Blade,Once honored defenders of Shurima against the ...,8,4,3,4,"[Fighter, Tank]",Blood Well,...,3.00,1.00,0.0,0.0,0,0,60,5.00,2.500,0.651
1,Ahri,103,the Nine-Tailed Fox,Innately connected to the magic of the spirit ...,3,4,8,5,"[Mage, Assassin]",Mana,...,2.50,0.60,8.0,0.8,0,0,53,3.00,2.000,0.668
2,Akali,84,the Rogue Assassin,Abandoning the Kinkou Order and her title of t...,5,3,8,7,[Assassin],Energy,...,9.00,0.90,50.0,0.0,0,0,62,3.30,3.200,0.625
3,Akshan,166,the Rogue Sentinel,"Raising an eyebrow in the face of danger, Aksh...",0,0,0,0,"[Marksman, Assassin]",Mana,...,3.75,0.65,8.2,0.7,0,0,52,3.50,4.000,0.638
4,Alistar,12,the Minotaur,Always a mighty warrior with a fearsome reputa...,6,9,5,7,"[Tank, Support]",Mana,...,8.50,0.85,8.5,0.8,0,0,62,3.75,2.125,0.625


In [6]:
# counter champions scraped from the leagueofgraphs website
champcounter_raw = pd.DataFrame(pd.read_csv('Counterchampion.csv'))
champcounter_raw.head()

,Champion,Unnamed: 1,Best Partner,Counters,Countered by
0,Aatrox,48.30%,Rammus,Nasus,Kled
1,Aatrox,NaN,Rammus+7.9%,Nasus+6.3%,Kled-9.3%
2,Top,NaN,NaN,NaN,NaN
3,Ahri,48.60%,Maokai,Veigar,Diana
4,Ahri,NaN,Maokai+6.2%,Veigar+5.9%,Diana-6.7%


In [7]:
# clean and merge onto champ_info_df
champcounter = []
for i in range(champcounter_raw.shape[0]):
  counterdict = {}
  if i % 3 == 0:
    counterdict["id"] = champcounter_raw.Champion[i]
    counterdict["Best_Partner"] = champcounter_raw["Best Partner"][i]
    counterdict["Counters"] = champcounter_raw["Counters "][i]
    counterdict["Countered_by"] = champcounter_raw["Countered by"][i]
    champcounter.append(counterdict)

df = pd.DataFrame(champcounter)
champ_info_df["Best_Partner"] = df["Best_Partner"]
champ_info_df["Counters"] = df["Counters"]
champ_info_df["Countered_by"] = df["Countered_by"]

champ_info_df.head()


,id,key,title,blurb,attack,defense,magic,difficulty,tags,partype,...,mpregenperlevel,crit,critperlevel,attackdamage,attackdamageperlevel,attackspeedperlevel,attackspeed,Best_Partner,Counters,Countered_by
0,Aatrox,266,the Darkin Blade,Once honored defenders of Shurima against the ...,8,4,3,4,"[Fighter, Tank]",Blood Well,...,0.0,0,0,60,5.00,2.500,0.651,Rammus,Nasus,Kled
1,Ahri,103,the Nine-Tailed Fox,Innately connected to the magic of the spirit ...,3,4,8,5,"[Mage, Assassin]",Mana,...,0.8,0,0,53,3.00,2.000,0.668,Maokai,Veigar,Diana
2,Akali,84,the Rogue Assassin,Abandoning the Kinkou Order and her title of t...,5,3,8,7,[Assassin],Energy,...,0.0,0,0,62,3.30,3.200,0.625,Pantheon,Diana,Fiora
3,Akshan,166,the Rogue Sentinel,"Raising an eyebrow in the face of danger, Aksh...",0,0,0,0,"[Marksman, Assassin]",Mana,...,0.7,0,0,52,3.50,4.000,0.638,Amumu,Talon,Yasuo
4,Alistar,12,the Minotaur,Always a mighty warrior with a fearsome reputa...,6,9,5,7,"[Tank, Support]",Mana,...,0.8,0,0,62,3.75,2.125,0.625,Kindred,Pyke,Renata Glasc


## Get match data

In [8]:
# test case example
me = watcher.summoner.by_name(my_region, 'Eric Norman')
my_ranked_stats = watcher.league.by_summoner(my_region, me['id'])

In [9]:
my_ranked_stats

[{'leagueId': '64de6776-701d-3731-9884-9f22c07f5e6b',
  'queueType': 'RANKED_SOLO_5x5',
  'tier': 'MASTER',
  'rank': 'I',
  'summonerId': 'z1-En7dbiCTdE1xUzgb2jPG730OvYULAADcav0Ae1_IBExw',
  'summonerName': 'Eric Norman',
  'leaguePoints': 74,
  'wins': 249,
  'losses': 245,
  'veteran': False,
  'inactive': False,
  'freshBlood': True,
  'hotStreak': True}]

In [10]:
my_ranked_stats[0]['rank'] 

'I'

In [11]:
# get champion dictionaries 
champ_dict_key_match = dict((int(champ['key']), champ['id']) for champ in current_champ_list['data'].values())
champ_dict_new_key = dict((index, champ['id']) for index, champ in enumerate(current_champ_list['data'].values()))

In [12]:
# get kda, recent top played champ, overall top played champs
def get_player_stats(puuid, summonerId, summonerName): 

  # total_kda = 0
  # history = watcher.match.matchlist_by_puuid(my_region, puuid)
  # history_df = pd.DataFrame(history, columns=['history'])
  # history_df['matchDetails'] = history_df['history'].apply(lambda x: watcher.match.by_id(my_region, x))
  # history_df['matchDetails'] = history_df['matchDetails'].apply(lambda x: x if x['info']['gameMode'] == 'CLASSIC' else np.nan)
  # history_df = history_df.dropna()
  # history_df = history_df.loc[0:4]
  # count = history_df.shape[0]
  # kda_champ_tuple = history_df.apply(lambda x: get_champ_and_kda(x['matchDetails'], summonerId), axis=1)
  # total_kda = kda_champ_tuple.apply(lambda x:x[0]).sum()
  # top_champ_last20_key = np.bincount(kda_champ_tuple.apply(lambda x:x[1])).argmax()
  # top_champ_last20 = champ_dict_key_match[top_champ_last20_key]
  # avg_kda = total_kda / count


    champion_mastery = watcher.champion_mastery.by_summoner(region=my_region, encrypted_summoner_id=summonerId)
    champion_mastery.sort(reverse=True, key=lambda x: x['championPoints'])
    top_champs_overall = list(map(lambda x: x['championId'], champion_mastery[0:3]))
    top_champs_overall = list(map(lambda x: champ_dict_key_match[x], top_champs_overall))

    player = watcher.summoner.by_name(my_region, summonerName)
    ranked_stats = watcher.league.by_summoner(my_region, player['id'])
    tier = np.nan
    wins = np.nan
    losses = np.nan 
    total = np.nan
    winPct = np.nan
    veteran = np.nan
    freshBlood = np.nan
    hotStreak = np.nan
    if (len(ranked_stats) > 0):
        
        tier = ranked_stats[0]['tier'] 
        wins = ranked_stats[0]['wins'] 
        losses = ranked_stats[0]['losses'] 
        total = wins + losses
        winPct = total and wins / total or 0
        veteran = ranked_stats[0]['veteran']
        freshBlood = ranked_stats[0]['freshBlood'] 
        hotStreak = ranked_stats[0]['hotStreak']

  # info = {"avg_kda": avg_kda, "top_champ_last20": top_champ_last20, "top_champs_overall": top_champs_overall}
    info = {"tier":tier,"top_champs_overall": top_champs_overall, "wins": wins, "losses": losses, "winPct": winPct, "veteran": veteran, "freshBlood":freshBlood, "hotStreak": hotStreak}
    return(info)

In [13]:
def get_player_stats(puuid, summonerId, summonerName):
    try:
        champion_mastery = watcher.champion_mastery.by_summoner(region=my_region, encrypted_summoner_id=summonerId)
        champion_mastery.sort(reverse=True, key=lambda x: x['championPoints'])
        top_champs_overall = list(map(lambda x: x['championId'], champion_mastery[0:3]))
        top_champs_overall = list(map(lambda x: champ_dict_key_match[x], top_champs_overall))

        player = watcher.summoner.by_name(my_region, summonerName)
        ranked_stats = watcher.league.by_summoner(my_region, player['id'])
        tier = np.nan
        wins = np.nan
        losses = np.nan 
        total = np.nan
        winPct = np.nan
        veteran = np.nan
        freshBlood = np.nan
        hotStreak = np.nan
        if (len(ranked_stats) > 0):
            tier = ranked_stats[0]['tier'] 
            wins = ranked_stats[0]['wins'] 
            losses = ranked_stats[0]['losses'] 
            total = wins + losses
            winPct = total and wins / total or 0
            veteran = ranked_stats[0]['veteran']
            freshBlood = ranked_stats[0]['freshBlood'] 
            hotStreak = ranked_stats[0]['hotStreak']

        info = {"tier":tier,"top_champs_overall": top_champs_overall, "wins": wins, "losses": losses, "winPct": winPct, "veteran": veteran, "freshBlood":freshBlood, "hotStreak": hotStreak}
    except:
        info = {"tier": np.nan, "top_champs_overall": [], "wins": np.nan, "losses": np.nan, "winPct": np.nan, "veteran": np.nan, "freshBlood": np.nan, "hotStreak": np.nan}

    return info


In [14]:
def singlegame_sort_visual(player_data):
  champion_list=[]
  champion_index=[]
  ##creates a champion list for the match-up
  for player in player_data:
    champion_list.append(player['championName'])

  participants = []
  ##trasnforms raw data from the nested dictionary into a dataframe, capturing 10 player_matchup vectors from a singlegame
  for i in range(5):
    player = player_data[i]
    champlist = champion_list.copy()
    participants_row = {}
    participants_row['champion'] = player['championName']
    champlist.remove(player['championName'])
    for index,champion in enumerate(champlist):
      participants_row['champion'+ str(index+1)] = champlist[index]
    participants_row['teamPosition'] = player['teamPosition']

    player_stats_dict = get_player_stats(player['puuid'], player['summonerId'], player['summonerName'])
    participants_row.update(player_stats_dict)

    participants_row['win'] = player['win']

    participants.append(participants_row)
    
  df = pd.DataFrame(participants)
  print('one game')
  return(df)  

In [15]:
player_list = ['COOKIEMONSTER123', 'Mei You Qing Xu', 'DinkyWinkyHigh', 'Malhari Mastani', 
 'mid gap rizz', 'Eästguy', 'Daimonz', 'Eric Norman', '971130', 'TheAugustGuy', 'DefinitelyNotAce', 'MeatBall7', 'DEFAULT NPC', 'Mevi','IClubNoobs', 
'Lurobelaud', 'Fferavalatha', 'Sharkive', 'Trash Cømpany', 'Cybershot', 'TTea', 'Kripsus', 'AMT', 'snake hoe', 'Phaśed', 'GFÞ', 'deep suppression', 'Jeanne Alter','6K OnFleek', 'Isles1', 'Fishlord', 'jamican banana', 
 'Jurassiq', 'HyperCarryGOD', 'Monchek', 'Zantimon', 'Zentinel', 'ningaflinga', 'Rivelía', 'deathbrain', 'Namssy', 'be like','Emilia Bublebee', 
'TrianglePlant', 'HarrisonTT',]

player_list_tot = ['COOKIEMONSTER123', 'Mei You Qing Xu', 'DinkyWinkyHigh', 'Malhari Mastani', 
 'mid gap rizz', 'Eästguy', 'Daimonz', 'Eric Norman', '971130', 'TheAugustGuy', 'DefinitelyNotAce', 'MeatBall7', 'DEFAULT NPC', 'Mevi','IClubNoobs', 
'Lurobelaud', 'Fferavalatha', 'Sharkive', 'Trash Cømpany', 'Cybershot', 'TTea', 'Kripsus', 'AMT', 'snake hoe', 'Phaśed', 'GFÞ', 'deep suppression', 'Jeanne Alter','6K OnFleek', 'Isles1', 'Fishlord', 'jamican banana', 
 'Jurassiq', 'HyperCarryGOD', 'Monchek', 'Zantimon', 'Zentinel', 'ningaflinga', 'Rivelía', 'deathbrain', 'Namssy', 'be like','Emilia Bublebee', 
'TrianglePlant', 'HarrisonTT','Chase', 'WayneDwops', 'dead dreams', 'UdonBdon', 'Princeeee', 'S8 IS SO FUN', 'G2 Juv', 'Daenethor', 
              'prohibit', 'TacoVaco', 'OkeyDoky', 'Vexxed04', 'black kid bard', 'Vanity', 'Trick2g', 'wavytoxic', 
              'Empress Bee', 'Bluebe1ry', 'crazybum', 'LLNHarkan123', 'LF UofT GFriend', 'DDOSS', 'KGFaker', 'ZxnithOP', 'perfunctory ToT'
              ,'Nilemars', 'Iattice']

In [17]:
match_df = pd.DataFrame()
for name in player_list: 
    player = watcher.summoner.by_name(my_region, name)
    history = watcher.match.matchlist_by_puuid(my_region, player['puuid'])
    history_df = pd.DataFrame(history, columns=['history'])
    history_df['match_details'] = history_df['history'].apply(lambda x: watcher.match.by_id(my_region, x))
    history_df['player_data'] = history_df['match_details'].apply(lambda x: x['info']['participants'])
    match_df = pd.concat([match_df, history_df], axis=0)
    match_df = match_df.reset_index(drop=True)
match_df

,history,match_details,player_data
0,NA1_4571308251,"{'metadata': {'dataVersion': '2', 'matchId': '...","[{'allInPings': 2, 'assistMePings': 4, 'assist..."
1,NA1_4571293790,"{'metadata': {'dataVersion': '2', 'matchId': '...","[{'allInPings': 0, 'assistMePings': 9, 'assist..."
2,NA1_4564017908,"{'metadata': {'dataVersion': '2', 'matchId': '...","[{'allInPings': 0, 'assistMePings': 0, 'assist..."
3,NA1_4563973675,"{'metadata': {'dataVersion': '2', 'matchId': '...","[{'allInPings': 0, 'assistMePings': 6, 'assist..."
4,NA1_4563953459,"{'metadata': {'dataVersion': '2', 'matchId': '...","[{'allInPings': 0, 'assistMePings': 0, 'assist..."
...,...,...,...
895,NA1_4590842027,"{'metadata': {'dataVersion': '2', 'matchId': '...","[{'allInPings': 0, 'assistMePings': 0, 'assist..."
896,NA1_4590827918,"{'metadata': {'dataVersion': '2', 'matchId': '...","[{'allInPings': 0, 'assistMePings': 0, 'assist..."
897,NA1_4590667256,"{'metadata': {'dataVersion': '2', 'matchId': '...","[{'allInPings': 0, 'assistMePings': 0, 'assist..."
898,NA1_4590632267,"{'metadata': {'dataVersion': '2', 'matchId': '...","[{'allInPings': 0, 'assistMePings': 0, 'assist..."


In [18]:
singlegame_sort_visual(match_df['player_data'][8])

one game


,champion,champion1,champion2,champion3,champion4,champion5,champion6,champion7,champion8,champion9,teamPosition,tier,top_champs_overall,wins,losses,winPct,veteran,freshBlood,hotStreak,win
0,Shen,Graves,Veigar,Varus,Nautilus,Gangplank,Diana,Qiyana,Ezreal,Karma,TOP,MASTER,"[Rengar, Riven, Gangplank]",72,69,0.510638,False,False,False,False
1,Graves,Shen,Veigar,Varus,Nautilus,Gangplank,Diana,Qiyana,Ezreal,Karma,JUNGLE,MASTER,"[Yasuo, Ekko, Graves]",187,189,0.497340,False,True,False,False
2,Veigar,Shen,Graves,Varus,Nautilus,Gangplank,Diana,Qiyana,Ezreal,Karma,MIDDLE,MASTER,"[Veigar, Gwen, Nasus]",286,265,0.519056,False,True,False,False
3,Varus,Shen,Graves,Veigar,Nautilus,Gangplank,Diana,Qiyana,Ezreal,Karma,BOTTOM,DIAMOND,"[Varus, Fizz, Azir]",48,61,0.440367,False,False,False,False
4,Nautilus,Shen,Graves,Veigar,Varus,Gangplank,Diana,Qiyana,Ezreal,Karma,UTILITY,DIAMOND,"[Nautilus, Karma, Pyke]",86,106,0.447917,False,False,False,False


In [150]:
summonerName = match_df['player_data'][8][3]['summonerName']
print(summonerName)
player = watcher.summoner.by_name(my_region, summonerName)
ranked_stats = watcher.league.by_summoner(my_region, player['id'])
ranked_stats

FloatingHagfish


[{'leagueId': '7a5785a2-a3c9-4040-8d96-72082e88e3b4',
  'queueType': 'RANKED_SOLO_5x5',
  'tier': 'DIAMOND',
  'rank': 'II',
  'summonerId': 'XvMSSFSAvKh4hFPuo82Wf_prqxvtKLk3LNl569cUeySy3XY',
  'summonerName': 'FloatingHagfish',
  'leaguePoints': 75,
  'wins': 48,
  'losses': 61,
  'veteran': False,
  'inactive': False,
  'freshBlood': False,
  'hotStreak': False}]

In [19]:
new_df = pd.DataFrame()
count=0
for player_data in match_df['player_data']: 
  df = singlegame_sort_visual(player_data)
  new_df = pd.concat([new_df, df], axis=0)
  count+=1 
  if count % 10 == 0 and count != 0 :
    print("10 games") 
  if count % 20 == 0 and count != 0 :
    print("20 games") 

print(count)

one game
one game
one game
one game
one game
one game
one game
one game
one game
one game
10 games
one game
one game
one game
one game
one game
one game
one game
one game
one game
one game
10 games
20 games
one game
one game
one game
one game
one game
one game
one game
one game
one game
one game
10 games
one game
one game
one game
one game
one game
one game
one game
one game
one game
one game
10 games
20 games
one game
one game
one game
one game
one game
one game
one game
one game
one game
one game
10 games
one game
one game
one game
one game
one game
one game
one game
one game
one game
one game
10 games
20 games
one game
one game
one game
one game
one game
one game
one game
one game
one game
one game
10 games
one game
one game
one game
one game
one game
one game
one game
one game
one game
one game
10 games
20 games
one game
one game
one game
one game
one game
one game
one game
one game
one game
one game
10 games
one game
one game
one game
one game
one game
one game
one game
one game
o

one game
one game
one game
one game
one game
one game
one game
10 games
20 games
one game
one game
one game
one game
one game
one game
one game
one game
one game
one game
10 games
one game
one game
one game
one game
one game
one game
one game
one game
one game
one game
10 games
20 games
one game
one game
one game
one game
one game
one game
one game
one game
one game
one game
10 games
one game
one game
one game
one game
one game
one game
one game
one game
one game
one game
10 games
20 games
one game
one game
one game
one game
one game
one game
one game
one game
one game
one game
10 games
one game
one game
one game
one game
one game
one game
one game
one game
one game
one game
10 games
20 games
one game
one game
one game
one game
one game
one game
one game
one game
one game
one game
10 games
one game
one game
one game
one game
one game
one game
one game
one game
one game
one game
10 games
20 games
one game
one game
one game
one game
one game
one game
one game
one game
one game
one game
1

In [20]:
new_df.shape

(4500, 20)

In [21]:
new_df.to_csv('df_4.csv', encoding = 'utf-8-sig') 